In [8]:
import pandas as pd
import numpy as np 
import os 
from tqdm import tqdm
from multiprocessing import Pool
import pickle
!ls

DataExploration.ipynb  pos2.csv			  test.csv
Model.ipynb	       posiou.csv		  test.csv.zip
new_04_train.csv       POS_TAG_IOU.ipynb	  train.csv
new_7_train.csv        reference.csv		  train.csv.zip
new_8_train.csv        sample_submission.csv.zip  Untitled.ipynb
pos1.csv	       Stemming.ipynb


In [2]:
import spacy
nlp=spacy.load("en_core_web_sm")

In [ ]:
df=pd.read_csv("new_8_train.csv")
df.shape
df.columns

In [4]:
def ner_tagger(q1,q2):
    
    doc1=nlp(q1.lower())
    out1=[(x.text,x.label_) for x in doc1.ents]
    
    doc2=nlp(q2.lower())
    out2=[(x.text,x.label_) for x in doc2.ents]
    
    return (out1,out2)

In [5]:
ref=zip(df['question1'],df['question2'])

out1=[]
out2=[]
for i in tqdm(ref):
    out=ner_tagger(i[0],i[1])
    out1.append(out[0])
    out2.append(out[1])

404287it [2:44:30, 40.96it/s]


In [9]:
with open("/home/ravi/Downloads/quora-question-pairs/ner1.dat","wb") as f:
    pickle.dump(out1,f)
    f.close()

In [10]:
with open("/home/ravi/Downloads/quora-question-pairs/ner2.dat","wb") as f:
    pickle.dump(out2,f)
    f.close()

In [19]:
def countEvents(x):
    out={}
    for i in x:
        if i not in out.keys():
            out[i[1]]=+1
        else:
            out[i[1]]+=1
    return out

In [27]:
df1=[]
for i in tqdm(out1):
    df1.append(countEvents(i))

df1=pd.DataFrame(df1)
df1.fillna(0,inplace=True)
df1.head(5)

columns=df1.columns
df1.columns=["q1"+x for x in columns]
df1.columns

Index(['q1GPE', 'q1CARDINAL', 'q1ORG', 'q1NORP', 'q1ORDINAL', 'q1PRODUCT',
       'q1PERSON', 'q1TIME', 'q1DATE', 'q1LANGUAGE', 'q1QUANTITY',
       'q1WORK_OF_ART', 'q1MONEY', 'q1LOC', 'q1FAC', 'q1EVENT', 'q1PERCENT',
       'q1LAW'],
      dtype='object')

In [31]:
df2=[]
for i in tqdm(out2):
    df2.append(countEvents(i))

df2=pd.DataFrame(df2)
df2.fillna(0,inplace=True)
df2.head(5)

columns=df2.columns
df2.columns=["q2"+x for x in columns]
df2.columns

Index(['q2NORP', 'q2ORG', 'q2ORDINAL', 'q2GPE', 'q2PERSON', 'q2CARDINAL',
       'q2TIME', 'q2DATE', 'q2LANGUAGE', 'q2MONEY', 'q2WORK_OF_ART',
       'q2QUANTITY', 'q2LOC', 'q2PRODUCT', 'q2LAW', 'q2EVENT', 'q2PERCENT',
       'q2FAC'],
      dtype='object')

In [47]:
def nerIOU(x,y):
    q1={}
    for i in x:
        if i[1] in q1.keys():
            q1[i[1]].append(i[0])
        else:
            q1[i[1]]=[i[0]]
    q2={}
    for i in y:
        if i[1] in q2.keys():
            q2[i[1]].append(i[0])
        else:
            q2[i[1]]=[i[0]]
    
    outputs={}
    for key in q1.keys():
        if key not in q2.keys():
            continue
        else:
            q1set=set(q1[key])
            q2set=set(q2[key])
            iou=len(q1set.intersection(q2set))/len(q1set.union(q2set))
            outputs[key]=iou
    return outputs
            

In [48]:
dfiou=[]
for x in tqdm(zip(out1,out2)):
    dfiou.append(nerIOU(x[0],x[1]))

404287it [00:00, 457794.17it/s]


In [50]:
dfiou=pd.DataFrame(dfiou)

In [53]:
dfiou.fillna(-1,inplace=True)

In [51]:
print(df1.shape)
print(df2.shape)
print(dfiou.shape)

(404287, 18)
(404287, 18)
(404287, 18)


In [55]:
print(dfiou.columns)
dfiou.columns=["iou"+x for x in dfiou.columns]
print(dfiou.columns)

Index(['ORG', 'ORDINAL', 'GPE', 'PERSON', 'CARDINAL', 'TIME', 'DATE', 'NORP',
       'LANGUAGE', 'MONEY', 'LOC', 'WORK_OF_ART', 'QUANTITY', 'EVENT',
       'PERCENT', 'LAW', 'PRODUCT', 'FAC'],
      dtype='object')
Index(['iouORG', 'iouORDINAL', 'iouGPE', 'iouPERSON', 'iouCARDINAL', 'iouTIME',
       'iouDATE', 'iouNORP', 'iouLANGUAGE', 'iouMONEY', 'iouLOC',
       'iouWORK_OF_ART', 'iouQUANTITY', 'iouEVENT', 'iouPERCENT', 'iouLAW',
       'iouPRODUCT', 'iouFAC'],
      dtype='object')


In [56]:
final_df=pd.concat([df1,df2,dfiou],axis=1)
print(final_df.shape)

(404287, 54)


In [77]:
q1=df.question1.iloc[1002]
print(q1)
doc1=nlp(q1.lower())
out1=[(x.text,x.label_) for x in doc1.ents]
print(out1)
q2=df.question2.iloc[1002]
print(q2)
doc2=nlp(q2.lower())
out2=[(x.text,x.label_) for x in doc2.ents]
print(out2)
print(final_df.iloc[1002])

Which is less healthy for your body, beer or Pepsi?
[('pepsi', 'ORG')]
Which is less healthy for your body light beer or Pepsi?
[('pepsi', 'ORG')]
q1GPE             0.0
q1CARDINAL        0.0
q1ORG             1.0
q1NORP            0.0
q1ORDINAL         0.0
q1PRODUCT         0.0
q1PERSON          0.0
q1TIME            0.0
q1DATE            0.0
q1LANGUAGE        0.0
q1QUANTITY        0.0
q1WORK_OF_ART     0.0
q1MONEY           0.0
q1LOC             0.0
q1FAC             0.0
q1EVENT           0.0
q1PERCENT         0.0
q1LAW             0.0
q2NORP            0.0
q2ORG             1.0
q2ORDINAL         0.0
q2GPE             0.0
q2PERSON          0.0
q2CARDINAL        0.0
q2TIME            0.0
q2DATE            0.0
q2LANGUAGE        0.0
q2MONEY           0.0
q2WORK_OF_ART     0.0
q2QUANTITY        0.0
q2LOC             0.0
q2PRODUCT         0.0
q2LAW             0.0
q2EVENT           0.0
q2PERCENT         0.0
q2FAC             0.0
iouORG            1.0
iouORDINAL       -1.0
iouGPE           

In [78]:
final_df.to_csv("NER.csv",index=None)